# Edge-Magic Total Labeling (EMTL) Tutorial

A hands-on tutorial for finding Edge-Magic Total Labelings using constraint programming.

## What You'll Learn
1. Mathematical definition of EMTL
2. Graph structure with partitions (A, B, C, D)
3. Using the solver to find labelings
4. Visualizing results
5. Exploring different parameters


## Setup

First, import the solver and required libraries.


In [ ]:
import sys
sys.path.insert(0, '..')

from emtl_solver import (
    solve_emtl,
    GraphParameters,
    GraphConstructor,
    EMTLVisualizer,
    SolverStatus,
)
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

print("Setup complete!")


## Mathematical Definition

An **Edge-Magic Total Labeling (EMTL)** of graph G = (V, E) is a bijection:

**f: V ∪ E → {1, 2, ..., |V| + |E|}**

such that for every edge uv:

**f(u) + f(uv) + f(v) = k** (constant magic sum)

### Our Graph Structure

The graph has four vertex partitions:
- **A**: m vertices (connected to all of B)
- **B**: n vertices (connected to A and C)
- **C**: n vertices (connected to B and D)  
- **D**: k vertices (connected to all of C)

The B-C subgraph is **t-regular** (each vertex has exactly t neighbors).


## Example 1: Basic Graph

Let's solve for a simple graph with parameters (m=2, n=2, k=2, t=1).


In [ ]:
# Solve for EMTL
result = solve_emtl(m=2, n=2, k=2, t=1, visualize=False, verbose=True)


In [ ]:
# Visualize the result
if result.exists:
    print(f"Magic constant k = {result.magic_constant}")
    fig = EMTLVisualizer.visualize(result, figsize=(14, 10), show=False)
    plt.show()
else:
    print(f"No EMTL found: {result.status.value}")


## Example 2: Larger Symmetric Graph

Now let's try a larger graph with symmetric parameters (3, 3, 3, 3).


In [ ]:
# Larger symmetric graph
result2 = solve_emtl(m=3, n=3, k=3, t=3, visualize=False, verbose=True)

if result2.exists:
    print(f"\nMagic constant k = {result2.magic_constant}")
    fig = EMTLVisualizer.visualize(result2, figsize=(16, 10), show=False)
    plt.show()


## Exploring Parameters

Let's see how different values of t (B-C regularity) affect the results.


In [ ]:
# Effect of regularity parameter t
print("Varying t (B-C regularity) with m=2, n=4, k=2")
print("=" * 50)
print(f"{'t':>3} {'Vertices':>10} {'Edges':>8} {'Status':>12} {'Magic k':>10}")
print("-" * 50)

m, n, k = 2, 4, 2
for t in range(n + 1):
    params = GraphParameters(m=m, n=n, k=k, t=t)
    result = solve_emtl(m=m, n=n, k=k, t=t, visualize=False, verbose=False, timeout=30)
    
    status = "Found" if result.exists else result.status.value
    magic = str(result.magic_constant) if result.exists else "-"
    print(f"{t:>3} {params.num_vertices:>10} {params.num_edges:>8} {status:>12} {magic:>10}")


## Verifying the Magic Property

Let's manually verify that all edge sums equal the magic constant.


In [ ]:
# Verify the magic property
result = solve_emtl(m=2, n=2, k=2, t=1, visualize=False, verbose=False)

if result.exists:
    print(f"Magic constant k = {result.magic_constant}\n")
    print("Verification of all edges:")
    print("-" * 50)
    
    for (u, v) in sorted(result.graph.edges()):
        label_u = result.vertex_labels[u]
        label_e = result.edge_labels[(u, v)]
        label_v = result.vertex_labels[v]
        total = label_u + label_e + label_v
        
        check = "✓" if total == result.magic_constant else "✗"
        print(f"{u}-{v}: {label_u} + {label_e} + {label_v} = {total} {check}")


## Try It Yourself

Modify the parameters below and run the cell to explore different graphs!


In [ ]:
# Try your own parameters!
m = 2  # vertices in set A
n = 3  # vertices in sets B and C
k = 2  # vertices in set D
t = 2  # B-C regularity (must be <= n)

result = solve_emtl(m=m, n=n, k=k, t=t, visualize=False, verbose=True)

if result.exists:
    fig = EMTLVisualizer.visualize(result, figsize=(14, 10), show=False)
    plt.show()
